## Imports

In [102]:
path = "../Data"
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pmp_functions_v4 as pmp

## Data Cleaning

In [103]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Equity Returns.xlsx",
    sheet_name = "WORLD - MXWO Index",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)

display(benchmark_data)

,Price (USD)
Date,
1970-01-30,94.2500
1970-02-27,96.9800
1970-03-31,97.0700
1970-04-30,87.8000
1970-05-29,82.0600
...,...
2025-07-31,8057.2246
2025-08-29,8269.7393
2025-09-30,8538.7900


In [104]:
# --- Load Factors Data ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100
display(factors_data)

# --- Riskfree Rate ---
riskfree = factors_data["RF"]
# riskfree = riskfree.to_frame()
display(riskfree)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_52685/84349411.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


,Mkt-RF,SMB,HML,RF
1926-07-31,0.0289,-0.0255,-0.0239,0.0022
1926-08-31,0.0264,-0.0114,0.0381,0.0025
1926-09-30,0.0038,-0.0136,0.0005,0.0023
1926-10-31,-0.0327,-0.0014,0.0082,0.0032
1926-11-30,0.0254,-0.0011,-0.0061,0.0031
...,...,...,...,...
2025-06-30,0.0486,0.0083,-0.0160,0.0034
2025-07-31,0.0198,0.0027,-0.0127,0.0034
2025-08-31,0.0184,0.0387,0.0442,0.0038
2025-09-30,0.0339,-0.0184,-0.0105,0.0033


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Name: RF, Length: 1192, dtype: float64

In [105]:
# --- Load Macro Data ---
monetary_data = pd.read_excel(
    f"{path}/2_year_yields.xlsx",
    index_col = 0,
    parse_dates = True
)
monetary_data.index = pd.to_datetime(monetary_data.index)
monetary_data.index = monetary_data.index + pd.offsets.MonthEnd(0)

display(monetary_data)

,US,UK,EU,JP,CH,AU
Date,,,,,,
1980-01-31,11.8790,NaN,NaN,NaN,NaN,NaN
1980-02-29,14.7390,NaN,NaN,NaN,NaN,NaN
1980-03-31,14.6390,NaN,NaN,NaN,NaN,NaN
1980-04-30,10.8190,NaN,NaN,NaN,NaN,NaN
1980-05-31,9.2590,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,3.7192,3.817,1.861,0.750,-0.048,3.2075
2025-07-31,3.9571,3.861,1.964,0.823,-0.082,3.3524
2025-08-31,3.6167,3.943,1.940,0.871,-0.103,3.3380


In [106]:
# --- Load Equity Prices ---
equity_map = {
    'US': 'US - SPX Index',
    'UK': 'UK - MXGB Index',
    'EU': 'EU - MXEM Index',
    'CH': 'CH - MXCH Index',
    'JP': 'JP - MXJP Index',
    'AU': 'AU - MXAU Index',
    'EM': 'EM - MXEF Index'
}

equity_price_list = []
for country, sheet in equity_map.items():
    df = pd.read_excel(
        f"{path}/Equity Returns.xlsx",
        sheet_name = sheet,
        index_col = 0,
        parse_dates = True
    )

    last_price = df.iloc[:, 0].resample('ME').last()
    last_price.name = country
    equity_price_list.append(last_price)

equity_prices = pd.concat(equity_price_list, axis=1)

display(equity_prices)

,US,UK,EU,CH,JP,AU,EM
Date,,,,,,,
1970-01-31,85.0200,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,89.7600,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,90.1511,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,82.2655,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,77.5125,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-07-31,30137.0645,10600.6826,488.1158,3991.8601,19.8688,6202.4824,2695.0002
2025-08-31,30747.9980,10995.8330,502.1103,4170.1406,21.2656,6451.0068,2734.5061
2025-09-30,31867.3594,11134.9863,518.8801,4160.9258,21.8248,6435.8457,2930.9309


In [107]:
# --- Load Currency Prices ---
currency_map = {
    'AU': 'AUDUSD Curncy',
    'JP': 'JPYUSD Curncy',
    'CH': 'CHFUSD Curncy',
    'EU': 'EURUSD Curncy',
    'UK': 'GBPUSD Curncy'
}

currency_list = []
for country, sheet in currency_map.items():
    df = pd.read_excel(
        f"{path}/FX.xlsx",
        sheet_name = sheet,
        index_col = 0,
        parse_dates = True
    )

    columns = ['Spot', 'FW Points']
    df = df[columns].copy()

    new_cols = {
        'Spot': f'{country} Spot',
        'FW Points': f'{country} Fwd Pts'
    }
    df.rename(columns=new_cols, inplace=True)
    currency_list.append(df)

currency_list = pd.concat(currency_list, axis = 1).resample('ME').last()
display(currency_list)

,AU Spot,AU Fwd Pts,JP Spot,JP Fwd Pts,CH Spot,CH Fwd Pts,EU Spot,EU Fwd Pts,UK Spot,UK Fwd Pts
Date,,,,,,,,,,
1971-01-31,NaN,NaN,NaN,NaN,0.2328,NaN,NaN,NaN,2.4174,NaN
1971-02-28,NaN,NaN,NaN,NaN,0.2322,NaN,NaN,NaN,2.4160,NaN
1971-03-31,NaN,NaN,NaN,NaN,0.2328,NaN,NaN,NaN,2.4168,NaN
1971-04-30,NaN,NaN,NaN,NaN,0.2328,NaN,NaN,NaN,2.4193,NaN
1971-05-31,NaN,NaN,NaN,NaN,0.2436,NaN,NaN,NaN,2.4178,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.6581,4.14,0.006943,-53.06,1.2609,-32.86,1.1787,25.69,1.3732,2.00
2025-07-31,0.6425,3.68,0.006634,-51.87,1.2311,-31.57,1.1415,23.52,1.3207,4.00
2025-08-31,0.6540,3.43,0.006800,-48.78,1.2492,-29.60,1.1686,22.54,1.3504,3.87


## Signal Generation

In [108]:
# --- Compute Monetary Policy Signal ---
monetary_signal = monetary_data.diff(12)
display(monetary_signal)

,US,UK,EU,JP,CH,AU
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.0343,-0.404,-0.972,0.384,-0.852,-0.9559
2025-07-31,-0.3004,0.035,-0.567,0.367,-0.742,-0.5227
2025-08-31,-0.2998,-0.166,-0.451,0.497,-0.693,-0.3335


In [109]:
# --- Equities Composite Signal Construction ---
# Logic: For Equities, we want decreasing Monetary Policy Yields (-)
monetary_policy_signal_equities = - monetary_signal

display(monetary_policy_signal_equities)

,US,UK,EU,JP,CH,AU
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.0343,0.404,0.972,-0.384,0.852,0.9559
2025-07-31,0.3004,-0.035,0.567,-0.367,0.742,0.5227
2025-08-31,0.2998,0.166,0.451,-0.497,0.693,0.3335


In [110]:
# --- Currencies Composite Signal Construction ---
# Logic: For Currencies, we want increasing Monetary Policy Yields (+)
monetary_policy_signal_currencies = monetary_signal

display(monetary_policy_signal_currencies)

,US,UK,EU,JP,CH,AU
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.0343,-0.404,-0.972,0.384,-0.852,-0.9559
2025-07-31,-0.3004,0.035,-0.567,0.367,-0.742,-0.5227
2025-08-31,-0.2998,-0.166,-0.451,0.497,-0.693,-0.3335


In [111]:
# --- Bonds Composite Signal Construction ---
# Logic: For Bonds, we want decreasing Monetary Policy Yields (-)
monetary_policy_signal_bonds = - monetary_signal

display(monetary_policy_signal_bonds)

,US,UK,EU,JP,CH,AU
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.0343,0.404,0.972,-0.384,0.852,0.9559
2025-07-31,0.3004,-0.035,0.567,-0.367,0.742,0.5227
2025-08-31,0.2998,0.166,0.451,-0.497,0.693,0.3335


In [112]:
# --- Interest Rates Composite Signal Construction ---
# Logic: For Interest Rates, we want decreasing Monetary Policy Yields (-)
monetary_policy_signal_int = - monetary_signal

display(monetary_policy_signal_int)

,US,UK,EU,JP,CH,AU
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.0343,0.404,0.972,-0.384,0.852,0.9559
2025-07-31,0.3004,-0.035,0.567,-0.367,0.742,0.5227
2025-08-31,0.2998,0.166,0.451,-0.497,0.693,0.3335


## Asset Class Returns

In [113]:
# --- Equity Returns ---
equity_returns = equity_prices.pct_change()
equity_returns = equity_returns.resample('ME').last()

display(equity_returns)

,US,UK,EU,CH,JP,AU,EM
Date,,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,0.055752,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,0.004357,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,-0.087471,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,-0.057776,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-07-31,0.022443,0.007016,-0.018617,-0.027445,-0.016742,-0.003080,0.020152
2025-08-31,0.020272,0.037276,0.028670,0.044661,0.070301,0.040069,0.014659
2025-09-30,0.036404,0.012655,0.033399,-0.002210,0.026296,-0.002350,0.071832


In [114]:
# --- Currency Returns ---
excess_returns_list = []
countries = ['AU', 'JP', 'CH', 'EU', 'UK'] 
divisor_map = {'JP': 100} # JPY uses 100, all others default to 10000

for country in countries:
    # 1. Get the relevant columns
    spot_col = f'{country} Spot'
    fwd_col = f'{country} Fwd Pts'

    # Determine the correct divisor (100 for JPY, 10000 for others)
    divisor = divisor_map.get(country, 10000)
    
    # 2. Calculate the Spot Return (EOM data, no shift needed)
    # Spot Return = (S_t / S_{t-1}) - 1
    spot_return = currency_list[spot_col].pct_change()

    # 3. Calculate the Funding Term (Cost of Carry)
    # Convert FW Pts (in BPS/Pips) to a decimal amount
    fwd_decimal_amount = currency_list[fwd_col] / divisor
    
    # We use Points_{t-1} because that was the price of carry agreed upon last month.
    # We divide by Spot_{t-1} to match the denominator of the spot_return.
    # Funding Term = (Fwd Decimal Amount) / Spot Price
    funding_term = fwd_decimal_amount.shift(1) / currency_list[spot_col].shift(1)
    
    # 4. Calculate the Excess Return
    # Note: If Points are Positive (Foreign Rate < US Rate), Funding Term is positive.
    # We SUBTRACT the funding cost (Paying the points).
    # If Points are Negative (Foreign Rate > US Rate), Funding Term is negative.
    # Subtracting a negative adds the yield (Earning the carry).
    # Excess Return = Spot Return - Funding Term
    excess_return = spot_return - funding_term
    
    # Rename and append
    excess_return.name = f'{country} Excess Return'
    excess_returns_list.append(excess_return)

# 5. Aggregate the Excess Returns DataFrame
excess_returns_df = pd.concat(excess_returns_list, axis=1)

# ADD THE US AS ZERO
# This ensures that if the US has the best macro data, 
# the model can allocate weight to "Cash" (USD).
excess_returns_df['US Excess Return'] = 0.0

# Drop the first row which contains NaN due to pct_change()
excess_returns_df.dropna(how='all', inplace=True)

# Display the resulting DataFrame
display(excess_returns_df)

,AU Excess Return,JP Excess Return,CH Excess Return,EU Excess Return,UK Excess Return,US Excess Return
Date,,,,,,
1971-01-31,NaN,NaN,NaN,NaN,NaN,0.0
1971-02-28,NaN,NaN,NaN,NaN,NaN,0.0
1971-03-31,NaN,NaN,NaN,NaN,NaN,0.0
1971-04-30,NaN,NaN,NaN,NaN,NaN,0.0
1971-05-31,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...
2025-06-30,0.022894,70.301023,0.039504,0.036845,0.020143,0.0
2025-07-31,-0.024334,76.377791,-0.021028,-0.033740,-0.038378,0.0
2025-08-31,0.017326,78.213144,0.017267,0.021680,0.022185,0.0


## Global Variables

In [115]:
frequency = 1
t_cost = 0
k = 2
weights_lag = 0
window = 12*20
short = True
beta_neutral = False
min_regions = 4
target_vol = 0.10
rf = riskfree
benchmark = benchmark_data

## Portfolio Construction

In [116]:
# --- Ranking & Weighting ---
# Rank countries 1 to N for each month based on the raw signal.
# axis = 1 means we rank across columns (countries).

# --- Equities ---
ranks_equities = monetary_policy_signal_equities.rank(axis = 1, method = 'average')
display(ranks_equities)

,US,UK,EU,JP,CH,AU
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,6.0,2.0,5.0,1.0,3.0,4.0
2025-07-31,3.0,2.0,5.0,1.0,6.0,4.0
2025-08-31,3.0,2.0,5.0,1.0,6.0,4.0


In [117]:
# --- Standardize Ranks ---
# Convert ranks into Z-scores (Weights) that sum to zero.
# Weight = (Rank - Mean_Rank) / Std_Dev_Rank

# --- Equities ---
rank_means_equities = ranks_equities.mean(axis = 1)
rank_stds_equities = ranks_equities.std(axis = 1)
standardized_weights_equities = ranks_equities.sub(rank_means_equities, axis = 0).div(rank_stds_equities, axis = 0)

display(standardized_weights_equities)

,US,UK,EU,JP,CH,AU
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.336306,-0.801784,0.801784,-1.336306,-0.267261,0.267261
2025-07-31,-0.267261,-0.801784,0.801784,-1.336306,1.336306,0.267261
2025-08-31,-0.267261,-0.801784,0.801784,-1.336306,1.336306,0.267261


In [118]:
# --- Volatility Scaling (Risk Management) ---
# Step A: Calculate 'Raw' Strategy Returns (Before Vol Scaling)
# IMPORTANT: Shift weights by 1 to trade next month's return.
strategy_raw_ret_equities = (standardized_weights_equities.shift(1) * equity_returns).sum(axis=1)

# Step B: Forecast Volatility
# Calculate realized volatility over a 36-month rolling window (annualized)
# We use the raw strategy's realized vol to estimate future volatility.
expected_vol_equities = strategy_raw_ret_equities.rolling(window = 36).std() * np.sqrt(12)

# We use previous rolling volatility (shift 1) to size today's position
lev_factor_equities = target_vol / expected_vol_equities.shift(1)

## **⭐ CRITICAL CHANGE: Scaling the Weights**

# Step C: Estimate Portfolio Weights
# Apply the leverage factor to the standardized weights
# We use .mul(axis=0) to multiply the 2D DataFrame (weights) 
# by the 1D Series (lev_factor_series) along the rows (axis=0).
final_strategy_weights_equities = standardized_weights_equities.mul(lev_factor_equities, axis=0)

display(final_strategy_weights_equities)

,US,UK,EU,JP,CH,AU
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-07-31,-0.150668,-0.452004,0.452004,-0.753340,0.753340,0.150668
2025-08-31,-0.153147,-0.459440,0.459440,-0.765733,0.765733,0.153147
2025-09-30,-0.152288,-0.456863,0.152288,-0.761438,0.761438,0.456863


# Backtest

In [119]:
results_equities = pmp.run_cc_strategy(
    weights = final_strategy_weights_equities,
    returns = equity_returns,
    rf = rf,
    frequency = frequency,
    t_cost = t_cost, 
    benchmark = benchmark,
    long_short = short,
    beta_neutral = beta_neutral
)

display(results_equities)

/Users/ryanyo/Developer/Backtesting for Portfolio Management/PMP/Macro Momentum/pmp_functions_v4.py:443: RuntimeWarning: invalid value encountered in scalar multiply
  tcost = turnover * t_cost
/Users/ryanyo/Developer/Backtesting for Portfolio Management/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:51: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/ryanyo/Developer/Backtesting for Portfolio Management/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:51: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_US,w_UK,w_EU,w_JP,w_CH,w_AU
Date,,,,,,,,,,,,
1970-02-28,0.000000,0.000000,0.0,0.000000,0.0,0.0062,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1970-03-31,0.000000,0.000000,0.0,0.000000,0.0,0.0057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1970-04-30,0.000000,0.000000,0.0,0.000000,0.0,0.0050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1970-05-31,0.000000,0.000000,0.0,0.000000,0.0,0.0053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1970-06-30,0.000000,0.000000,0.0,0.000000,0.0,0.0058,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-31,0.021493,0.021493,0.0,0.900358,0.0,0.0034,0.566527,-0.339021,0.325138,-0.550035,-0.110944,0.108335
2025-08-31,-0.020223,-0.020223,0.0,1.033484,0.0,0.0038,-0.105974,-0.328090,0.330100,-0.565936,0.557073,0.112827
2025-09-30,-0.018232,-0.018232,0.0,0.378319,0.0,0.0033,-0.111144,-0.329982,0.341125,-0.558874,0.547629,0.111246


# Performance Statistics

In [120]:
pmp.run_perf_summary_benchmark_vs_strategy(results_equities, alreadyXs= False)

,Benchmark,Strategy
Arithm Avg Total Return,0.0000,NaN
Arithm Avg Xs Return,-4.3370,NaN
Std Xs Returns,0.9533,NaN
Sharpe Arithmetic,-4.5497,NaN
Geom Avg Total Return,0.0000,NaN
Geom Avg Xs Return,-4.4196,NaN
Sharpe Geometric,-4.6363,NaN
Min Xs Return,-1.3500,NaN
Max Xs Return,0.0000,NaN
Skewness,-0.6489,NaN


In [121]:
factor_data = factors_data[["Mkt-RF", "SMB", "HML"]]
display(factors_data)
pmp.run_factor_regression(results_equities, factor_data, alreadyXs = False)

,Mkt-RF,SMB,HML,RF
1926-07-31,0.0289,-0.0255,-0.0239,0.0022
1926-08-31,0.0264,-0.0114,0.0381,0.0025
1926-09-30,0.0038,-0.0136,0.0005,0.0023
1926-10-31,-0.0327,-0.0014,0.0082,0.0032
1926-11-30,0.0254,-0.0011,-0.0061,0.0031
...,...,...,...,...
2025-06-30,0.0486,0.0083,-0.0160,0.0034
2025-07-31,0.0198,0.0027,-0.0127,0.0034
2025-08-31,0.0184,0.0387,0.0442,0.0038
2025-09-30,0.0339,-0.0184,-0.0105,0.0033


MissingDataError: exog contains inf or nans